In [1]:
import torch
import torch.nn as nn
import csv
import json
import math
from nltk import word_tokenize as tokenize
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool

In [2]:
import sys
import json
doc_name = "data2/val.txt.src"
summ_name = "data2/val.txt.tgt.tagged"
task_name = "cnn"
val_size = 13368

capitalize = False

vocab_name = "preprocessing-cnn-all/vocab.json"
continue_from = "models/covModel9" # if none, put None
eval_dir = "evaluation-cnn/"
data_seq_name = eval_dir+'tmp.json'


num_threads = 4
vocab = json.load(open(vocab_name, 'r'))
vocab_inv = {ind:word for word, ind in vocab.items()}
print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


In [3]:
HIDDEN_DIM = 256
EMB_DIM = 50
INPUT_MAX = 400
OUTPUT_MAX = 100
epsilon = 1e-10
VOC_SIZE = len(vocab)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda


In [5]:
def getlines(name):
    total = !wc -l {name}
    return int(total[0].split()[0])

In [6]:
summaries = []
articles = []

# Reader = csv.reader(open(train_name, newline='', encoding='ISO-8859-1'), delimiter=',')
with open(doc_name, newline='', encoding='utf-8') as f:
    total = getlines(doc_name)
    for i,line in tqdm(enumerate(f), total=total):
        line = line.strip()
        articles.append(line)
with open(summ_name, newline='', encoding='utf-8') as f:
    total = getlines(summ_name)
    for i,line in tqdm(enumerate(f), total=total):        
        line = line.replace("<t>", "").replace("</t>", "")
        line = line.strip()
        summaries.append(line)
        
print(len(summaries))


13368


In [7]:
summaries = summaries[:val_size]
articles = articles[:val_size]
print(len(summaries))

13368


In [8]:
def get_dig(n):
    return 1 if n < 10 else (get_dig(n/10) + 1)

digits = get_dig(val_size)
print(digits)

def make_gold(summ):
    for i,s in tqdm(enumerate(summ), total=len(summ)):
        filename = (eval_dir+'gold/'+task_name+'.A.{:0'+str(digits)+'d}.txt').format(i+1)
#         print(filename)
        with open(filename, 'w') as f:
            f.write(s)
# make_gold(summaries)

5


In [9]:
data = []
if num_threads > 1:
    def task(s):
        words = tokenize(s)
        return ['<bos>'] + words + ['<eos>']
    
    with Pool(num_threads) as p:
        chunksize = 20
        summaries = list(tqdm(p.imap(task, summaries, chunksize=chunksize), total=len(summaries)))
        articles = list(tqdm(p.imap(task, articles, chunksize=chunksize), total=len(articles)))
    data = summaries+articles
else:
    for i, summ in tqdm(enumerate(summaries+articles), total=len(summaries+articles)):
        words = tokenize(summ)
        data.append(['<bos>'] + words + ['<eos>'])

In [10]:
num_summ = len(summaries)
index = 0
print("[summary]", summaries[index])
print("[documen]", articles[index])

[summary] ['<bos>', 'a', 'man', 'in', 'suburban', 'boston', 'is', 'selling', 'snow', 'online', 'to', 'customers', 'in', 'warmer', 'states', '.', 'for', '$', '89', ',', 'he', 'will', 'ship', '6', 'pounds', 'of', 'snow', 'in', 'an', 'insulated', 'styrofoam', 'box', '.', '<eos>']
[documen] ['<bos>', '-lrb-', 'cnn', '-rrb-', 'the', 'only', 'thing', 'crazier', 'than', 'a', 'guy', 'in', 'snowbound', 'massachusetts', 'boxing', 'up', 'the', 'powdery', 'white', 'stuff', 'and', 'offering', 'it', 'for', 'sale', 'online', '?', 'people', 'are', 'actually', 'buying', 'it', '.', 'for', '$', '89', ',', 'self-styled', 'entrepreneur', 'kyle', 'waring', 'will', 'ship', 'you', '6', 'pounds', 'of', 'boston-area', 'snow', 'in', 'an', 'insulated', 'styrofoam', 'box', '--', 'enough', 'for', '10', 'to', '15', 'snowballs', ',', 'he', 'says', '.', 'but', 'not', 'if', 'you', 'live', 'in', 'new', 'england', 'or', 'surrounding', 'states', '.', '``', 'we', 'will', 'not', 'ship', 'snow', 'to', 'any', 'states', 'in', 

In [11]:
seqdata = {'summary':[], 'document':[]}
num_summ = len(data) / 2
for words in tqdm(summaries, total=num_summ):    
    seq = []
    for w in words:        
        try:
            wid = vocab[w]
        except KeyError:
            wid = vocab["<unk>"]
        seq.append(wid)
    seqdata['summary'].append(seq)

for words in tqdm(articles, total=num_summ):    
    seq = []
    for w in words:
        try:
            wid = vocab[w]
        except KeyError:
            wid = vocab["<unk>"]
        seq.append(wid)
    seqdata['document'].append(seq)
json.dump(seqdata, open(data_seq_name, "w"))

In [12]:
from torch.utils import data

class Dataset(data.Dataset):
    
    def __init__(self, data_name, vocab):
        data = json.load(open(data_name, 'r'))
        sum_list = data['summary']
        data_list = data['document']
        # idata -> list
        self.size = len(sum_list)
        self.dataset = []
        self.sum_len = 0
        
        for i in range(len(sum_list)):
            if(len(data_list[i]) <= INPUT_MAX):
                data = [vocab['<pad>']]*(INPUT_MAX-len(data_list[i])) + data_list[i]
            else:
                data = data_list[i][:INPUT_MAX]
                
            if(len(sum_list[i]) <= OUTPUT_MAX):
                sum_in = sum_list[i] + [vocab['<pad>']]*(OUTPUT_MAX-len(sum_list[i]))
            else:
                sum_in = sum_list[i][:OUTPUT_MAX]
                
            sum_out_raw = sum_list[i][1:]
            if(len(sum_out_raw) <= OUTPUT_MAX):
                sum_out = sum_out_raw + [vocab['<pad>']]*(OUTPUT_MAX-len(sum_out_raw))
            else:
                sum_out = sum_out_raw[:OUTPUT_MAX]
                
            self.dataset.append([data, sum_in, sum_out])
     
    def __len__(self):
        return self.size
    def __getitem__(self, index):
        # output ari sum sumout
#         print([vocab['<bos>']]+self.dataset[index][1]+[vocab['<eos>']])
        return (torch.tensor(self.dataset[index][0]),\
                torch.tensor(self.dataset[index][1]),\
                torch.tensor(self.dataset[index][2]))

In [13]:
from PointerGenerator import PointerGenerator
if str(device) == 'cpu':
    saved_model = torch.load(continue_from, map_location=lambda storage, location: storage)
else:
    saved_model = torch.load(continue_from)

inf_model = PointerGenerator(HIDDEN_DIM, EMB_DIM, INPUT_MAX, OUTPUT_MAX, VOC_SIZE).to(device)
inf_model.load_state_dict(saved_model['model'])

In [14]:
inf_model

PointerGenerator(
  (emb_layer): Embedding(90000, 50)
  (encoder): LSTM(50, 256, batch_first=True, bidirectional=True)
  (decoder): LSTM(50, 512, batch_first=True)
  (attention_softmax): Softmax()
  (pro_layer): Sequential(
    (0): Linear(in_features=1024, out_features=90000, bias=True)
    (1): Softmax()
  )
  (pgen_layer): Sequential(
    (0): Linear(in_features=1074, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [15]:
def readable(sent):
    try:
        end = sent.index('<eos>')
    except ValueError:
        end = len(sent)
    sent = " ".join(sent[:end])
    sent = sent.replace("<bos>", '')
    sent = sent.replace("<eos>", '')
    sent = sent.replace("<unk>", '-UNK-')
    sent = sent.replace("<pad>", '')
    if capitalize:
        sent = sent.capitalize()
    return sent

In [16]:
testing_set = Dataset(data_seq_name, vocab)
examples = min(100000, len(testing_set))
print(examples)
params2 = {'batch_size':16,
           'shuffle':False,
         'num_workers': 4}
testing_generator = data.DataLoader(testing_set, **params2)

# t_bos = torch.tensor([vocab['<bos>']]*OUTPUT_MAX).repeat(params2['batch_size'], 1).to(device)

total = int(math.ceil(examples / params2['batch_size']))
for i, (in1, in2, out) in tqdm(enumerate(testing_generator), total= total):
    in1, in2, out = in1.to(device), in2.to(device), out.to(device)

    predict = inf_model.inference(in1, vocab['<bos>']);

    for j, (sent_tensor, outtensor) in enumerate(zip(predict, out)):
        sent = [vocab_inv[wid.item()] for wid in sent_tensor]
#         sent2 = [vocab_inv[wid.item()] for wid in outtensor]
        
        numeric = i*params2['batch_size'] + j + 1
        
        filename = (eval_dir+'mine/'+task_name+'.{:0'+str(digits)+'d}.txt').format(numeric)
        
        with open(filename, 'w') as mine:
            mine.write(readable(sent))

    if i == total:
        break
    


13368


[info] Training mode detected. Switched to evaluation mode.



In [17]:
in1.device

device(type='cuda', index=0)

In [18]:

from pyrouge import Rouge155

r = Rouge155()
r.system_dir = eval_dir+'mine/'
r.model_dir = eval_dir+'gold/'
r.system_filename_pattern = task_name+'.(\d+).txt'
r.model_filename_pattern = task_name+'.[A-Z].#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

2019-06-03 19:36:07,570 [MainThread  ] [INFO ]  Writing summaries.
2019-06-03 19:36:07,572 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmpefqv77l7/system and model files to /tmp/tmpefqv77l7/model.
2019-06-03 19:36:07,573 [MainThread  ] [INFO ]  Processing files in evaluation-cnn/mine/.
2019-06-03 19:36:07,589 [MainThread  ] [INFO ]  Processing cnn.01668.txt.
2019-06-03 19:36:07,590 [MainThread  ] [INFO ]  Processing cnn.12307.txt.
2019-06-03 19:36:07,591 [MainThread  ] [INFO ]  Processing cnn.05047.txt.
2019-06-03 19:36:07,593 [MainThread  ] [INFO ]  Processing cnn.00746.txt.
2019-06-03 19:36:07,594 [MainThread  ] [INFO ]  Processing cnn.07470.txt.
2019-06-03 19:36:07,595 [MainThread  ] [INFO ]  Processing cnn.12559.txt.
2019-06-03 19:36:07,596 [MainThread  ] [INFO ]  Processing cnn.11614.txt.
2019-06-03 19:36:07,598 [MainThread  ] [INFO ]  Processing cnn.03118.txt.
2019-06-03 19:36:07,599 [MainThread  ] [INFO ]  Processing cnn.01796.txt.
2019-06-03 19:36:

2019-06-03 19:36:07,721 [MainThread  ] [INFO ]  Processing cnn.13351.txt.
2019-06-03 19:36:07,721 [MainThread  ] [INFO ]  Processing cnn.13105.txt.
2019-06-03 19:36:07,722 [MainThread  ] [INFO ]  Processing cnn.08774.txt.
2019-06-03 19:36:07,723 [MainThread  ] [INFO ]  Processing cnn.05277.txt.
2019-06-03 19:36:07,724 [MainThread  ] [INFO ]  Processing cnn.01896.txt.
2019-06-03 19:36:07,725 [MainThread  ] [INFO ]  Processing cnn.13012.txt.
2019-06-03 19:36:07,725 [MainThread  ] [INFO ]  Processing cnn.06332.txt.
2019-06-03 19:36:07,726 [MainThread  ] [INFO ]  Processing cnn.02903.txt.
2019-06-03 19:36:07,727 [MainThread  ] [INFO ]  Processing cnn.03151.txt.
2019-06-03 19:36:07,727 [MainThread  ] [INFO ]  Processing cnn.12105.txt.
2019-06-03 19:36:07,728 [MainThread  ] [INFO ]  Processing cnn.08920.txt.
2019-06-03 19:36:07,729 [MainThread  ] [INFO ]  Processing cnn.01075.txt.
2019-06-03 19:36:07,729 [MainThread  ] [INFO ]  Processing cnn.00005.txt.
2019-06-03 19:36:07,730 [MainThread  ]

2019-06-03 19:36:07,796 [MainThread  ] [INFO ]  Processing cnn.06346.txt.
2019-06-03 19:36:07,796 [MainThread  ] [INFO ]  Processing cnn.03427.txt.
2019-06-03 19:36:07,797 [MainThread  ] [INFO ]  Processing cnn.00216.txt.
2019-06-03 19:36:07,797 [MainThread  ] [INFO ]  Processing cnn.06828.txt.
2019-06-03 19:36:07,798 [MainThread  ] [INFO ]  Processing cnn.11396.txt.
2019-06-03 19:36:07,798 [MainThread  ] [INFO ]  Processing cnn.07538.txt.
2019-06-03 19:36:07,799 [MainThread  ] [INFO ]  Processing cnn.09284.txt.
2019-06-03 19:36:07,800 [MainThread  ] [INFO ]  Processing cnn.04583.txt.
2019-06-03 19:36:07,800 [MainThread  ] [INFO ]  Processing cnn.10593.txt.
2019-06-03 19:36:07,801 [MainThread  ] [INFO ]  Processing cnn.04553.txt.
2019-06-03 19:36:07,801 [MainThread  ] [INFO ]  Processing cnn.02350.txt.
2019-06-03 19:36:07,802 [MainThread  ] [INFO ]  Processing cnn.00729.txt.
2019-06-03 19:36:07,803 [MainThread  ] [INFO ]  Processing cnn.04221.txt.
2019-06-03 19:36:07,803 [MainThread  ]

2019-06-03 19:36:07,864 [MainThread  ] [INFO ]  Processing cnn.05420.txt.
2019-06-03 19:36:07,865 [MainThread  ] [INFO ]  Processing cnn.11640.txt.
2019-06-03 19:36:07,866 [MainThread  ] [INFO ]  Processing cnn.05038.txt.
2019-06-03 19:36:07,866 [MainThread  ] [INFO ]  Processing cnn.00011.txt.
2019-06-03 19:36:07,867 [MainThread  ] [INFO ]  Processing cnn.00981.txt.
2019-06-03 19:36:07,867 [MainThread  ] [INFO ]  Processing cnn.06852.txt.
2019-06-03 19:36:07,868 [MainThread  ] [INFO ]  Processing cnn.12023.txt.
2019-06-03 19:36:07,868 [MainThread  ] [INFO ]  Processing cnn.00117.txt.
2019-06-03 19:36:07,869 [MainThread  ] [INFO ]  Processing cnn.11399.txt.
2019-06-03 19:36:07,869 [MainThread  ] [INFO ]  Processing cnn.12169.txt.
2019-06-03 19:36:07,870 [MainThread  ] [INFO ]  Processing cnn.03122.txt.
2019-06-03 19:36:07,871 [MainThread  ] [INFO ]  Processing cnn.08745.txt.
2019-06-03 19:36:07,871 [MainThread  ] [INFO ]  Processing cnn.08581.txt.
2019-06-03 19:36:07,872 [MainThread  ]

2019-06-03 19:36:07,940 [MainThread  ] [INFO ]  Processing cnn.02251.txt.
2019-06-03 19:36:07,941 [MainThread  ] [INFO ]  Processing cnn.11422.txt.
2019-06-03 19:36:07,941 [MainThread  ] [INFO ]  Processing cnn.06674.txt.
2019-06-03 19:36:07,942 [MainThread  ] [INFO ]  Processing cnn.04574.txt.
2019-06-03 19:36:07,942 [MainThread  ] [INFO ]  Processing cnn.06230.txt.
2019-06-03 19:36:07,943 [MainThread  ] [INFO ]  Processing cnn.06407.txt.
2019-06-03 19:36:07,944 [MainThread  ] [INFO ]  Processing cnn.02728.txt.
2019-06-03 19:36:07,944 [MainThread  ] [INFO ]  Processing cnn.10145.txt.
2019-06-03 19:36:07,945 [MainThread  ] [INFO ]  Processing cnn.09357.txt.
2019-06-03 19:36:07,945 [MainThread  ] [INFO ]  Processing cnn.08627.txt.
2019-06-03 19:36:07,946 [MainThread  ] [INFO ]  Processing cnn.05325.txt.
2019-06-03 19:36:07,946 [MainThread  ] [INFO ]  Processing cnn.06359.txt.
2019-06-03 19:36:07,947 [MainThread  ] [INFO ]  Processing cnn.09117.txt.
2019-06-03 19:36:07,947 [MainThread  ]

...

2019-06-03 19:37:40,462 [MainThread  ] [INFO ]  Processing cnn.A.07893.txt.
2019-06-03 19:37:40,471 [MainThread  ] [INFO ]  Processing cnn.A.03819.txt.
2019-06-03 19:37:40,478 [MainThread  ] [INFO ]  Processing cnn.A.11278.txt.
2019-06-03 19:37:40,480 [MainThread  ] [INFO ]  Processing cnn.A.10798.txt.
2019-06-03 19:37:40,482 [MainThread  ] [INFO ]  Processing cnn.A.12601.txt.
2019-06-03 19:37:40,484 [MainThread  ] [INFO ]  Processing cnn.A.10327.txt.
2019-06-03 19:37:40,485 [MainThread  ] [INFO ]  Processing cnn.A.11443.txt.
2019-06-03 19:37:40,497 [MainThread  ] [INFO ]  Processing cnn.A.04214.txt.
2019-06-03 19:37:40,502 [MainThread  ] [INFO ]  Processing cnn.A.01694.txt.
2019-06-03 19:37:40,510 [MainThread  ] [INFO ]  Processing cnn.A.09570.txt.
2019-06-03 19:37:40,515 [MainThread  ] [INFO ]  Processing cnn.A.11199.txt.
2019-06-03 19:37:40,522 [MainThread  ] [INFO ]  Processing cnn.A.07341.txt.
2019-06-03 19:37:40,528 [MainThread  ] [INFO ]  Processing cnn.A.11237.txt.
2019-06-03 1

2019-06-03 19:37:41,072 [MainThread  ] [INFO ]  Processing cnn.A.09885.txt.
2019-06-03 19:37:41,085 [MainThread  ] [INFO ]  Processing cnn.A.12687.txt.
2019-06-03 19:37:41,087 [MainThread  ] [INFO ]  Processing cnn.A.06273.txt.
2019-06-03 19:37:41,090 [MainThread  ] [INFO ]  Processing cnn.A.00103.txt.
2019-06-03 19:37:41,100 [MainThread  ] [INFO ]  Processing cnn.A.08328.txt.
2019-06-03 19:37:41,106 [MainThread  ] [INFO ]  Processing cnn.A.00505.txt.
2019-06-03 19:37:41,108 [MainThread  ] [INFO ]  Processing cnn.A.06226.txt.
2019-06-03 19:37:41,110 [MainThread  ] [INFO ]  Processing cnn.A.07416.txt.
2019-06-03 19:37:41,113 [MainThread  ] [INFO ]  Processing cnn.A.00789.txt.
2019-06-03 19:37:41,119 [MainThread  ] [INFO ]  Processing cnn.A.07030.txt.
2019-06-03 19:37:41,125 [MainThread  ] [INFO ]  Processing cnn.A.10933.txt.
2019-06-03 19:37:41,130 [MainThread  ] [INFO ]  Processing cnn.A.11452.txt.
2019-06-03 19:37:41,139 [MainThread  ] [INFO ]  Processing cnn.A.03448.txt.
2019-06-03 1

2019-06-03 19:37:41,685 [MainThread  ] [INFO ]  Processing cnn.A.03912.txt.
2019-06-03 19:37:41,689 [MainThread  ] [INFO ]  Processing cnn.A.07541.txt.
2019-06-03 19:37:41,694 [MainThread  ] [INFO ]  Processing cnn.A.07145.txt.
2019-06-03 19:37:41,704 [MainThread  ] [INFO ]  Processing cnn.A.07028.txt.
2019-06-03 19:37:41,706 [MainThread  ] [INFO ]  Processing cnn.A.01675.txt.
2019-06-03 19:37:41,718 [MainThread  ] [INFO ]  Processing cnn.A.03702.txt.
2019-06-03 19:37:41,730 [MainThread  ] [INFO ]  Processing cnn.A.06104.txt.
2019-06-03 19:37:41,740 [MainThread  ] [INFO ]  Processing cnn.A.01913.txt.
2019-06-03 19:37:41,751 [MainThread  ] [INFO ]  Processing cnn.A.03841.txt.
2019-06-03 19:37:41,758 [MainThread  ] [INFO ]  Processing cnn.A.11014.txt.
2019-06-03 19:37:41,767 [MainThread  ] [INFO ]  Processing cnn.A.00175.txt.
2019-06-03 19:37:41,776 [MainThread  ] [INFO ]  Processing cnn.A.02495.txt.
2019-06-03 19:37:41,780 [MainThread  ] [INFO ]  Processing cnn.A.02243.txt.
2019-06-03 1

2019-06-03 19:37:42,326 [MainThread  ] [INFO ]  Processing cnn.A.07572.txt.
2019-06-03 19:37:42,328 [MainThread  ] [INFO ]  Processing cnn.A.06229.txt.
2019-06-03 19:37:42,337 [MainThread  ] [INFO ]  Processing cnn.A.05980.txt.
2019-06-03 19:37:42,344 [MainThread  ] [INFO ]  Processing cnn.A.06343.txt.
2019-06-03 19:37:42,345 [MainThread  ] [INFO ]  Processing cnn.A.02790.txt.
2019-06-03 19:37:42,350 [MainThread  ] [INFO ]  Processing cnn.A.07532.txt.
2019-06-03 19:37:42,352 [MainThread  ] [INFO ]  Processing cnn.A.01031.txt.
2019-06-03 19:37:42,355 [MainThread  ] [INFO ]  Processing cnn.A.03277.txt.
2019-06-03 19:37:42,357 [MainThread  ] [INFO ]  Processing cnn.A.06178.txt.
2019-06-03 19:37:42,360 [MainThread  ] [INFO ]  Processing cnn.A.11114.txt.
2019-06-03 19:37:42,369 [MainThread  ] [INFO ]  Processing cnn.A.00601.txt.
2019-06-03 19:37:42,380 [MainThread  ] [INFO ]  Processing cnn.A.04136.txt.
2019-06-03 19:37:42,385 [MainThread  ] [INFO ]  Processing cnn.A.00126.txt.
2019-06-03 1

---------------------------------------------
1 ROUGE-1 Average_R: 0.31309 (95%-conf.int. 0.31106 - 0.31517)
1 ROUGE-1 Average_P: 0.38768 (95%-conf.int. 0.38507 - 0.39022)
1 ROUGE-1 Average_F: 0.33463 (95%-conf.int. 0.33255 - 0.33652)
---------------------------------------------
1 ROUGE-2 Average_R: 0.12700 (95%-conf.int. 0.12522 - 0.12872)
1 ROUGE-2 Average_P: 0.15747 (95%-conf.int. 0.15528 - 0.15950)
1 ROUGE-2 Average_F: 0.13562 (95%-conf.int. 0.13377 - 0.13738)
---------------------------------------------
1 ROUGE-3 Average_R: 0.07075 (95%-conf.int. 0.06921 - 0.07228)
1 ROUGE-3 Average_P: 0.08806 (95%-conf.int. 0.08619 - 0.08987)
1 ROUGE-3 Average_F: 0.07554 (95%-conf.int. 0.07394 - 0.07708)
---------------------------------------------
1 ROUGE-4 Average_R: 0.04609 (95%-conf.int. 0.04473 - 0.04748)
1 ROUGE-4 Average_P: 0.05777 (95%-conf.int. 0.05611 - 0.05941)
1 ROUGE-4 Average_F: 0.04924 (95%-conf.int. 0.04780 - 0.05059)
---------------------------------------------
1 ROUGE-L Aver

In [19]:
# model3(1200): 0.26311, 0.10240, 0.19684
# model0(400): 0.26328, 0.10495, 0.19771
# model1(400): 0.26385, 0.10600, 0.19873
# model2(400): 0.27241, 0.11119, 0.20404
# model3(400): 0.27178, 0.11072, 0.20360

In [20]:
# import matplotlib.pyplot as plt
# rouge1 = [0.26328, 0.26385, 0.27241, 0.27178]
# rouge2 = [0.10495, 0.10600, 0.11119, 0.11072]
# rougel = [0.19771, 0.19873, 0.20404, 0.20360]

# plt.plot(rouge1, label='rouge-1')
# plt.xlabel('epoch')
# plt.ylabel('f1')
# plt.title('testing ROUGE')

# plt.legend(loc='upper right')
# plt.show()